In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import networkx as nx

from mpl_toolkits.mplot3d import Axes3D
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
#getting our data

%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training

node2vec_sparse = pd.read_csv("sparse_node2vec.csv")
word2vec = pd.read_csv("word2vec_features.csv")
pages_df = pd.read_csv("node_classification.csv")
training_graph = pd.read_csv("training_graph.csv")

%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Test
node2vec_test = pd.read_csv("testing_node2vec.csv")
test_edges = pd.read_csv("test_edges.csv")

/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Training
/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Test


In [17]:
#removing artifacts from conversion to csv
w2v = word2vec.drop(["Unnamed: 0"], axis=1)
n2v = node2vec_sparse.drop(["Unnamed: 0"], axis=1)
pages_df.drop(["id"], axis=1)
pages = np.array(pages_df)
pages = pages[:, 1]

In [18]:
#creating a full concatenated data matrix
finlist = pd.concat([n2v, w2v], axis=1)
print(finlist.head)

<bound method NDFrame.head of               0         1         2  ...        97        98        99
0      0.000000  0.000000  0.000000  ... -0.050624  1.482534  0.193179
1     -0.221936 -0.618475  0.157521  ...  0.296546  1.296658  0.388548
2      0.218412 -0.916328 -0.851940  ... -0.043109 -0.913542 -0.020716
3     -0.024505  0.328660  0.539155  ... -0.297195  0.703305  0.082358
4     -0.012618 -0.605725 -0.121598  ... -0.477068  0.999462 -0.210479
...         ...       ...       ...  ...       ...       ...       ...
22465 -0.171671 -0.376132 -0.101205  ...  0.297647  1.092919  0.081616
22466  0.328381  0.097097 -0.310297  ...  0.288717  1.778235 -0.180114
22467  0.000000  0.000000  0.000000  ...  0.067341  1.230007  0.292623
22468  0.042153 -0.027671 -0.397323  ... -0.120122  0.793197 -0.209848
22469  0.106251 -0.725376 -0.350690  ... -0.007161  0.849211 -0.007069

[22470 rows x 200 columns]>


In [23]:
#go through each edge pair
edges = np.array(training_graph)
finlist = np.array(finlist)
subtracted_squared = np.zeros((len(edges), 201))
for i in range(len(edges)):
  #find the squared difference
  temp = np.subtract(finlist[edges[i,0]], finlist[edges[i,1]])
  temp = np.square(temp)

  #add to new matrix
  subtracted_squared[i,:200] = temp

  #add whether or not they have the same page type
  if pages[edges[i, 0]] == pages[edges[i,1]]:
    subtracted_squared[i, 200] = 0
  else:
    subtracted_squared[i, 200] = 1

In [26]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training
isolated_nodes = pd.read_csv("isolated_nodes.csv")

#generating our false pairs of edges
non_edges = np.zeros((len(edges),2), dtype=np.int64)
isolated = np.array(isolated_nodes)
i = 0
print(edges.shape)
while i < len(edges):
  #get a random pair of nodes, add if not isolated
  temp = np.random.randint(0, 22470, size=(2,), dtype=np.int64)
  if temp[0] not in isolated:
    if temp[1] not in isolated:
      non_edges[i] = temp
      i += 1

/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Training
(132038, 2)


In [30]:
subtracted_squared_non = np.zeros((len(non_edges), 201))
for i in range(len(non_edges)):
  #find the squared difference
  temp = np.subtract(finlist[non_edges[i,0]], finlist[non_edges[i,1]])
  temp = np.square(temp)

  #add to a new matrix
  subtracted_squared_non[i,:200] = temp

  #add whether or not they have the same page
  if pages[non_edges[i, 0]] == pages[non_edges[i,1]]:
    subtracted_squared_non[i, 200] = 0
  else:
    subtracted_squared_non[i, 200] = 1

In [32]:
#generate our matrices for ML

X = np.zeros((len(edges)*2, 201))
X[:len(edges),:] = subtracted_squared
X[len(edges):,:] = subtracted_squared_non
y = np.zeros((len(edges)*2,), dtype=np.int8)
y[len(edges):] = 1

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [34]:
#scale and split data for training
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [35]:
# create KNN classifier
knn = KNeighborsClassifier(n_neighbors=9, weights='distance')

# train the model using the training sets
knn.fit(X_train, y_train)

# predict the response for test dataset
y_pred = knn.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8967926385943653


In [43]:
#same procedure, go through test edges
test_edges = np.array(test_edges)
subtracted_squared_test = np.zeros((len(test_edges), 201))
for i in range(len(test_edges)):
  #find squared difference between pair of nodes
  temp = np.subtract(finlist[test_edges[i,0]], finlist[test_edges[i,1]])
  temp = np.square(temp)

  #add to new matrix
  subtracted_squared_test[i,:200] = temp

  #add 1 if they share same page label, 0 if not
  if pages[edges[i, 0]] == pages[edges[i,1]]:
    subtracted_squared[i, 200] = 0
  else:
    subtracted_squared[i, 200] = 1

In [44]:
#fit and predict with all of our data and the testing data
knn_test = KNeighborsClassifier(n_neighbors=9, weights='distance')
knn_test.fit(X_scaled,y)
predictions = knn.predict(subtracted_squared_test)

In [45]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Test
test_labels_df = pd.read_csv('test_labels.csv')

#check accurcay
test_labels = np.array(test_labels_df)
metrics.accuracy_score(test_labels, predictions)

/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Test


0.46598893706084615

In [46]:
from tensorflow.keras.models import Sequential
import tensorflow as tf

#define our model
def trainModel(model,epochs,optimizer):
    batch_size=128
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics='accuracy')
    return model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size)
model=tf.keras.models.Sequential([
    tf.keras.layers.Dense(512,activation='relu',input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1,activation='sigmoid'),
])

#training and predicting on model for validation
print(model.summary())
model_history=trainModel(model=model,epochs=20,optimizer='adam')
model.predict(X_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               103424    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8

array([[0.9999932 ],
       [0.01382661],
       [0.01981947],
       ...,
       [0.03025246],
       [0.01325658],
       [0.7864114 ]], dtype=float32)

In [47]:
#predictions from the actual testing data
nn_predictions = np.round_(model.predict(subtracted_squared_test)).astype(np.int8)

In [48]:
#accuracy for the actual testing data OUCH
metrics.accuracy_score(test_labels, nn_predictions)

0.4293616385109882